# QUBO formulation of polynomial equation

In this notebook we show how to use `QUBOPS` to solve simple non linear system of equations. We start by defining our system of equations as follows:

$$
 0.5 - x_0 + x_1 = 0 \\
 1 - x_1 = 0 \\
 3 - x_0^2 - x_2 = 0 \\
 -0.5 x_1^2 + x_2 - x_3 = 0 
$$

We have therefore 4 variables and 4 equations.

## Classical Newton Raphson solution 

We first solve this system using a newton raphson approach. We have implemented this approach in our own package `quantum_newton_raphson` but other implementation are available. To use the newton raphson solver we need to define a function that returns the value of the non linear system for a given input. 


In [1]:
import numpy as np

def nlfunc(input):
    x0,x1,x2,x3 = input

    def f0():
        return 1/2 - x0 + x1
    
    def f1():
        return 1 - x1
    
    def f2():
        return 3  - x0**2  - x2

    def f3():
        return - 0.5*x1**2 + x2 - x3
    
    return np.array([f0(), f1(), f2(), f3()])

We can now solve the system using our own library. Note that with the default values, our quantum newton raphson library uses a purely classical approach. 

In [2]:
from quantum_newton_raphson.newton_raphson import newton_raphson

initial_point = np.random.rand(4)
res = newton_raphson(nlfunc, initial_point)
assert np.allclose(nlfunc(res.solution), 0)

/home/nico/QuantumApplicationLab/QuantumNewtonRaphson/quantum_newton_raphson/utils.py:74: SparseEfficiencyWarning: spsolve requires A be CSC or CSR matrix format
  warn("spsolve requires A be CSC or CSR matrix format", SparseEfficiencyWarning)


The solution of the system is therefore

In [3]:
ref_sol = res.solution
ref_sol

array([1.5 , 1.  , 0.75, 0.25])

## Polynomial equation 

We first write the polynomial equation as follow (https://www.nature.com/articles/s41598-019-46729-0) 

$$
F(X) = 0
$$

with

$$
F_i = P_i^{(0)} + \sum_j P_{ij}^{(1)}x_j + \sum_{jk} P_{ijk}^{(2)}x_j x_k  + \sum_{jkl} P_{ijkl}^{(3)}x_j x_k x_l = 0
$$

To solve the system we minimize the residual sum of square

$$
\chi^2 = [P^{(0)} + P^{(1)} X + P^{(2)} X^2 + P^{(3)} X^3]^2
$$


We need to define the matrices $P_{ij...}^{(n)}$ for our system of equation

In [4]:
import sparse
def define_matrices():
    
    # system of equations
    num_equations = 4
    num_variables = 4


    P0 = np.zeros(num_equations)
    P0[0] = 1/2
    P0[1] = 1
    P0[2] = 3
    P0[3] = 0

    P1 = np.zeros((num_equations, num_variables))
    P1[0, 0] = -1
    P1[0, 1] =  1

    P1[1, 1] = -1

    P1[2, 2] = -1

    P1[3, 2] =  1
    P1[3, 3] = -1
   

    P2 = np.zeros((num_equations, num_variables, num_variables))
    P2[2, 0, 0] = -1
    P2[3, 1, 1] = -1/2

    return sparse.COO(P0), sparse.COO(P1), sparse.COO(P2)
matrices = define_matrices()

## Encoding

As for any QUBO approach, we need to encode the variables we are looking for in a series of qubit. Several encodings are possible. We here decide to use differnet encodings for the first and last two variables. We also decide to use here the `SteepestDescentSampler` to be able to run that code locally. Quantum solvers are available through the Leap cloud service.


In [5]:
from qubops.qubops_mixed_vars import QUBOPS_MIXED
from qubops.encodings import PositiveQbitEncoding
from qubops.solution_vector import SolutionVector_V2 as SolutionVector
from qubops.mixed_solution_vector import MixedSolutionVector_V2 as MixedSolutionVector
from dwave.samplers import SteepestDescentSolver


# define the encoding for the first two varialbes
nqbit = 5
step = 0.05
encoding1 = PositiveQbitEncoding(nqbit = nqbit, step=step, offset=0, var_base_name='x')
sol_vec1 = SolutionVector(2, encoding=encoding1)

# define the encoding for the alst two variables 
nqbit = 4
step = 0.05
encoding2 = PositiveQbitEncoding(nqbit = nqbit, step=step, offset=0, var_base_name='x')
sol_vec2 = SolutionVector(2, encoding=encoding2)

# define the solution vector
sol_vec = MixedSolutionVector([sol_vec1,sol_vec2])

# instantiat the QUBOPS solver
options = {'num_reads':10000, 'sampler':SteepestDescentSolver()}
qubo = QUBOPS_MIXED(sol_vec, options)

We can check which values are possible for the different encodings:

In [6]:
np.sort(encoding1.get_possible_values())

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75, 0.8 , 0.85, 0.9 , 0.95, 1.  , 1.05,
       1.1 , 1.15, 1.2 , 1.25, 1.3 , 1.35, 1.4 , 1.45, 1.5 , 1.55])

In [7]:
np.sort(encoding2.get_possible_values())

array([0.  , 0.05, 0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 , 0.45, 0.5 ,
       0.55, 0.6 , 0.65, 0.7 , 0.75])

## Solving the system

We can now solve the system and analyze the solution

In [8]:
sol = qubo.solve(matrices, strength=1E5)
sol = np.array(sol).reshape(-1)

data_ref, eref = qubo.compute_energy(ref_sol)
data_sol, esol = qubo.compute_energy(sol)


np.set_printoptions(precision=3)

print('prec: ', encoding1.get_average_precision(), encoding2.get_average_precision())
print('\n')

print('ref : ', np.array(ref_sol)) 
print('sol : ', sol)
print('diff: ', ref_sol - sol)
print('\n')

print('encoded_ref: ', np.array(data_ref[0]))
print('encoded_sol: ', np.array(data_sol[0]))
print('diff       : ', np.array(data_ref[0]) - np.array(data_sol[0]))
print('\n')
print('eref: ', eref)
print('esol: ', esol)
print('\n')
print('res_ref: ', np.linalg.norm(nlfunc(data_ref[0])))
print('res_sol: ', np.linalg.norm(nlfunc(data_sol[0])))



prec:  0.05 0.05


ref :  [1.5  1.   0.75 0.25]
sol :  [1.5  1.   0.75 0.25]
diff:  [ 0.000e+00 -1.110e-16  2.294e-11  2.733e-11]


encoded_ref:  [1.5  1.   0.75 0.25]
encoded_sol:  [1.5  1.   0.75 0.25]
diff       :  [0. 0. 0. 0.]


eref:  -10.250000000116417
esol:  -10.250000000116417


res_ref:  0.0
res_sol:  0.0
